In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


###This is to install the pydrive for google to work on witht the files and provied the linkage of the different python files


In [0]:
#!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


###This is to get the extract other files that are utitlsed as a part of preprocessing and utilizing the model for training .


In [0]:
your_module = drive.CreateFile({'id':'1zTGvr3r8OO1YLzDKoVQnD8yXH9PqPnx1'})
model_module =  drive.CreateFile({'id':'1_cNErAl9ozHbyxYgvBalzDt4YSXN_Zie'})

In [0]:
your_module.GetContentFile('get_data.py')
model_module.GetContentFile('cnnmodel.py') 

###Get the required libraries 

In [5]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from get_data import load_fer2013
from get_data import preprocess_input
from cnnmodel import mini_XCEPTION
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


### Providing the parameters 

In [0]:
batch_size = 32
num_epochs = 10000
input_shape = (48, 48, 1)
validation_split = .2
verbose = 1
num_classes = 7
patience = 50

###Used so that all the call backs and the model stats can be stored, So here a folder is created in the google drive .

In [0]:
base_path = "drive/Colab Notebooks/models/"

In [0]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

### Defining the model  and its parameters 

In [9]:
model = mini_XCEPTION(input_shape, num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


W0729 20:14:25.836499 140206003541888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0729 20:14:25.861692 140206003541888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 20:14:25.868590 140206003541888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0729 20:14:25.900434 140206003541888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0729 20:14:25.901829 1402060035

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 46, 46, 8)    72          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 46, 46, 8)    32          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 46, 46, 8)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

###Making the necessary callbacks  , refer  , https://keras.io/callbacks/

In [0]:
log_file_path = '_emotions.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
trained_models_path = '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]


###Loading the dataset and running the model

In [0]:
faces, emotions = load_fer2013()
faces = preprocess_input(faces)
num_samples, num_classes = emotions.shape
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)
model.fit_generator(data_generator.flow(xtrain, ytrain,
                                            batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))


/content/get_data.py:35: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  emotions = pd.get_dummies(dataset['emotion']).as_matrix()
W0729 20:15:00.482325 140206003541888 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10000
898/897 [==============================] - 320s 356ms/step - loss: 1.7907 - acc: 0.3143 - val_loss: 1.6217 - val_acc: 0.4037

Epoch 00001: val_loss improved from inf to 1.62171, saving model to _mini_XCEPTION.01-0.40.hdf5
Epoch 2/10000
608/897 [===================>..........] - ETA: 1:32 - loss: 1.5439 - acc: 0.4186